In [1]:
"""
# 전처리 배분

1(성준) - 0~9,   40~49, 80~89, 120, 
       - credit_card_balance.csv
       - POS_CASH_balance.csv
       - previous_application.csv
       - installments_payments.csv
2(장호) - 10~19, 50~59, 90~99, 121
3(두영) - 20~29, 60~69, 100~109, 122, 
       - bureau.csv,
4(혜빈) - 30~39, 70~79, 110~119, 123, 
       - bureau_balance.csv
5(준혁) -  
"""

"""
# 데이터 목록
application_test.csv
application_train.csv
credit_card_balance.csv
POS_CASH_balance.csv
previous_application.csv
installments_payments.csv
bureau.csv
bureau_balance.csv
sample_submission.csv
"""

'\n# 데이터 목록\napplication_test.csv\napplication_train.csv\nPOS_CASH_balance.csv\nbureau.csv\nbureau_balance.csv\ncredit_card_balance.csv\ninstallments_payments.csv\nprevious_application.csv\nsample_submission.csv\n'

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
import user_functions as uf

import platform
import matplotlib
%matplotlib inline
# 한글 출력되게 설정
uf.korean()

## "POS_CASH_balance.csv"
---

In [157]:
pos_cash = pd.read_csv(uf.get_f_path()+"POS_CASH_balance.csv")

In [158]:
pos_cash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SK_ID_PREV             int64  
 1   SK_ID_CURR             int64  
 2   MONTHS_BALANCE         int64  
 3   CNT_INSTALMENT         float64
 4   CNT_INSTALMENT_FUTURE  float64
 5   NAME_CONTRACT_STATUS   object 
 6   SK_DPD                 int64  
 7   SK_DPD_DEF             int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB


In [229]:
# # "MONTHS_BALANCE" # 잔액 갱신일
# # 큰 문제 없음
# # uf.check_feature(pos_cash, "MONTHS_BALANCE")
# # uf.box_plot(pos_cash, "MONTHS_BALANCE")

# # "CNT_INSTALMENT" 할부 기간
# # 0 처리
# # uf.check_feature(pos_cash, "CNT_INSTALMENT")
# # uf.box_plot(pos_cash, "CNT_INSTALMENT")
# pos_cash["CNT_INSTALMENT"].fillna(0, inplace=True)

# # "CNT_INSTALMENT_FUTURE" 잔여 할부 기간
# # 0 처리
# # uf.check_feature(pos_cash, "CNT_INSTALMENT_FUTURE")
# # uf.box_plot(pos_cash, "CNT_INSTALMENT_FUTURE")
# pos_cash["CNT_INSTALMENT_FUTURE"].fillna(0, inplace=True)

# # "NAME_CONTRACT_STATUS" 
# # "XNA" 항목 잔여 할부기간이 남아있으므로 Active로 대체
# # uf.check_feature(pos_cash, "NAME_CONTRACT_STATUS")
# pos_cash.loc[pos_cash["NAME_CONTRACT_STATUS"] == "XNA", "NAME_CONTRACT_STATUS"] = "Active"

In [226]:
# SK_DPD(연체 경과일) - SK_DPD_DEF(연체 경과일(소액 연체 제외)) = 소액 연체 경과일
pos_cash["소액연체_경과일"] = pos_cash["SK_DPD"] - pos_cash["SK_DPD_DEF"]

# 연체 관련 지표만 따로 추출함
temp_df = pos_cash[["SK_ID_CURR", "SK_DPD", "SK_DPD_DEF", "소액연체_경과일"]]

# 동일인의 총 연체일수를 더함
temp_merge = temp_df.groupby(["SK_ID_CURR"]).sum()
temp_merge.reset_index(inplace=True)

# 소액연체 비율
temp_merge["소액연체_비율"] = temp_merge["소액연체_경과일"] / temp_merge["SK_DPD"]
# 소액연체 비율이 결측값이면 0으로 대체함. 왜냐하면 연체 한 적이 없기 때문에 결측임
temp_merge["소액연체_비율"].fillna(0, inplace=True)

# index = SK_ID_CURR
temp_merge.set_index("SK_ID_CURR", inplace=True)

# 소액연체비율이랑 전체 연체 경과일만 가져옴
temp_merge.drop(columns=["소액연체_경과일", "SK_DPD_DEF"], inplace=True)

# 저장
temp_merge.to_csv("../pre_data/new_POS_CASH_balance.csv")

---